**This file contains the code for the final predictions from my meta model(LGBM regressor)**

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
file_1=open('/content/drive/MyDrive/Project Energy Consumption/df_te_red_final_modified.txt','rb')
df_tr_red_final=pickle.load(file_1)

**Dropping the features which are not important**

In [ ]:
df_tr_red_final.drop(['cloud_coverage','sea_level_pressure','wind_direction','wind_speed',
                      'is_summer_month','is_pub_holiday'],axis=1,inplace=True)

In [ ]:
df_tr_red_final.drop('row_id',axis=1,inplace=True)

**Here I will be importing all the 3 base models(LGBM,XGBOOST AND CATBOOST) and my meta model(LGBM).The predictions from my meta model on the test set will work as an input for my final meta model.**

In [ ]:
file_2=open('/content/drive/MyDrive/Project Energy Consumption/meta_model.txt','rb')
meta_model=pickle.load(file_2)

In [ ]:
file_2=open('/content/drive/MyDrive/Project Energy Consumption/lgb_model_enesemble.txt','rb')
lgbm_model=pickle.load(file_2)

In [ ]:
file_2=open('/content/drive/MyDrive/Project Energy Consumption/xgb_model_ensemble.txt','rb')
xgb_model=pickle.load(file_2)

[09:54:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
pip install catboost

     |████████████████████████████████| 65.8MB 48kB/s 


In [ ]:
file_2=open('/content/drive/MyDrive/Project Energy Consumption/catboost_model_ensemble.txt','rb')
catboost_model=pickle.load(file_2)

In [ ]:
test_pred_lgbm=lgbm_model.predict(df_tr_red_final)

In [ ]:
df_lgbm=pd.DataFrame(test_pred_lgbm,columns=['lgb_pred'])

In [ ]:
test_pred_catboost=catboost_model.predict(df_tr_red_final)

In [ ]:
df_catboost=pd.DataFrame(test_pred_catboost,columns=['cat_pred'])

In [ ]:
test_pred_xgb=xgb_model.predict(df_tr_red_final)

In [ ]:
df_xgb=pd.DataFrame(test_pred_xgb,columns=['xgb_pred'])

**Concatenating the data from my base model and using it as input to make my final predictions.**

In [ ]:
df_final=pd.concat([df_lgbm,df_catboost,df_xgb],axis=1)

In [ ]:
y_test=meta_model.predict(df_final)

**Converting the predictions back to normal meter readings by taking the exponent as we did the log transformation initially.**

In [ ]:
y_test=np.expm1(y_test)

In [ ]:
y_test_round=np.round(y_test,4)

In [ ]:
y_test_df=pd.DataFrame(y_test_round)

In [ ]:
y_test_df.rename(columns={0:'meter_reading'},inplace=True)

In [ ]:
y_test_df['row_id']=y_test_df.index

In [ ]:
y_test_df=y_test_df[['row_id','meter_reading']]

In [ ]:
y_test_df['row_id']=y_test_df['row_id'].astype(int)

In [ ]:
for i in tqdm(y_test_df[y_test_df['meter_reading']<0].index):
  y_test_df['meter_reading'][i]=0

  0%|          | 0/73481 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 73481/73481 [00:05<00:00, 13275.70it/s]


In [ ]:
y_test_df.to_csv('final_ensemble.csv',index=False,header=True)

**Final predicted meter readings**

In [ ]:
y_test_df.head(10)

,row_id,meter_reading
0,0,148.5249
1,1,43.6769
2,2,15.4947
3,3,252.0610
4,4,795.5073
5,5,73.8136
6,6,81.4485
7,7,794.4527
8,8,481.4482
9,9,322.5558


                                                             **End of Notebook**